In [11]:
import sys, os
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, PROJECT_ROOT)
try:
    from config.config import POLYGON_API_KEY
except ImportError:
    print("Error: config.py not found or POLYGON_API_KEY not set.")
    print("Please create config.py and add your Polygon API key.")
    POLYGON_API_KEY = None # Or raise an error

API_KEY = POLYGON_API_KEY

In [13]:
import requests
import json

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [14]:

API_KEY = API_KEY
ticker = "IOT"
url = f"https://api.polygon.io/v2/reference/news?apiKey={API_KEY}&ticker={ticker}"

response = requests.get(url)
data = response.json()
print(data)

{'results': [{'id': 'e878b5624997ad21aa3d4e349d27f5e34e8613db60cd30c7752300d5881686d0', 'publisher': {'name': 'GlobeNewswire Inc.', 'homepage_url': 'https://www.globenewswire.com', 'logo_url': 'https://s3.polygon.io/public/assets/news/logos/globenewswire.svg', 'favicon_url': 'https://s3.polygon.io/public/assets/news/favicons/globenewswire.ico'}, 'title': 'Samsara ($SAMS) Unveils AI-Powered Workforce Automation on the Blockchain', 'author': 'N/A', 'published_utc': '2025-03-24T20:00:00Z', 'article_url': 'https://www.globenewswire.com/news-release/2025/03/24/3048280/0/en/Samsara-SAMS-Unveils-AI-Powered-Workforce-Automation-on-the-Blockchain.html', 'tickers': ['IOT'], 'image_url': 'https://ml.globenewswire.com/Resource/Download/1efdcfa9-5af0-4e71-8d76-e2b3b5b99738', 'description': 'Samsara, a Web3 platform, enables businesses to create, deploy, and monetize AI agents through a no-code interface and blockchain integration, revolutionizing operational efficiency and customer engagement.', 'k

In [15]:
ticker = "AAPL"  # You can change this ticker as needed
start_date = "2025-01-01"
end_date = "2025-02-10"
limit = 200
order = 'desc'

url = (
          f"https://api.polygon.io/v2/reference/news?"
          f"apiKey={API_KEY}&ticker={ticker}"
          f"&published_utc.gte={start_date}"
          f"&published_utc.lte={end_date}"
          f"&limit={limit}"
          f"&order={order}"
          )
response = requests.get(url)
data = response.json()
#data = pd.json_normalize(data['results'])
data

{'results': [{'id': 'ea61f1cbe55f6eb2d11f3dcf7bf8d25b2bc62926c1521d39700cebc6e90e51f4',
   'publisher': {'name': 'Benzinga',
    'homepage_url': 'https://www.benzinga.com/',
    'logo_url': 'https://s3.polygon.io/public/assets/news/logos/benzinga.svg',
    'favicon_url': 'https://s3.polygon.io/public/assets/news/favicons/benzinga.ico'},
   'title': 'China Sets Sights On Apple, Nvidia And Other US Tech Companies As Trade Tensions Rise',
   'author': 'Adam Eckert',
   'published_utc': '2025-02-10T17:14:47Z',
   'article_url': 'https://www.benzinga.com/news/global/25/02/43600852/china-sets-sights-on-us-tech-companies-as-trade-tensions-rise',
   'tickers': ['AAPL', 'NVDA', 'GOOG', 'GOOGL', 'XLK', 'MAGS'],
   'image_url': 'https://cdn.benzinga.com/files/images/story/2025/02/10/Apple-Shenzhen-China-media-preview-store.jpeg?width=1200&height=800&fit=crop',
   'description': 'China is targeting U.S. tech companies like Apple, Nvidia, and Alphabet in an effort to gain leverage in trade negotiat

In [13]:
# Load the JSON data into a Python list
#data = json.loads(data)

# Create a DataFrame where each row is one ticker record with its sentiment info.
df = pd.DataFrame(data)

# Optionally, if you want one-hot encoding for the sentiment column (in case there are other sentiment types),
# you can use pd.get_dummies:
#df_sentiment_dummies = pd.get_dummies(df['results'], prefix='sentiment')

# Concatenate the original DataFrame with the one-hot encoded sentiment columns
#df_final = pd.concat([df, df_sentiment_dummies], axis=1)

df = pd.json_normalize(df['results'])


# Define a helper function to flatten the nested list if necessary.
def flatten_insights(x):
    if isinstance(x, list):
        # If the first element is a list, flatten one level.
        if len(x) > 0 and isinstance(x[0], list):
            return [item for sublist in x for item in sublist]
    return x

# Apply the helper function to create a flat version of insights.
df['insights_flat'] = df['insights'].apply(flatten_insights)

# Use explode() to create one row per insight.
df_exploded = df.explode('insights_flat').reset_index(drop=True)

# Use pd.json_normalize on the exploded column to turn each dictionary into columns.
insights_expanded = pd.json_normalize(df_exploded['insights_flat'])

# Optionally, if you want to keep article-level info (like "article"), join it.
df_final = df_exploded.drop(columns=['insights', 'insights_flat']).reset_index(drop=True).join(insights_expanded)


df_final

,id,title,author,published_utc,article_url,tickers,image_url,description,keywords,publisher.name,publisher.homepage_url,publisher.logo_url,publisher.favicon_url,amp_url,ticker,sentiment,sentiment_reasoning
0,0d66b060785f30e8a5725598b00435d5232c88f0f161d1...,Discover the Future: IoT-Based Asset Tracking ...,Globe Newswire,2025-01-23T02:30:00Z,https://www.globenewswire.com/news-release/202...,"[SMTC, IOT, SQNS]",https://ml.globenewswire.com/Resource/Download...,The global IoT-based asset tracking and monito...,"[IoT, asset tracking, monitoring, manufacturin...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,SMTC,positive,Semtech Corporation is mentioned as an industr...
1,0d66b060785f30e8a5725598b00435d5232c88f0f161d1...,Discover the Future: IoT-Based Asset Tracking ...,Globe Newswire,2025-01-23T02:30:00Z,https://www.globenewswire.com/news-release/202...,"[SMTC, IOT, SQNS]",https://ml.globenewswire.com/Resource/Download...,The global IoT-based asset tracking and monito...,"[IoT, asset tracking, monitoring, manufacturin...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,IOT,positive,Samsara Inc. is listed as one of the key playe...
2,0d66b060785f30e8a5725598b00435d5232c88f0f161d1...,Discover the Future: IoT-Based Asset Tracking ...,Globe Newswire,2025-01-23T02:30:00Z,https://www.globenewswire.com/news-release/202...,"[SMTC, IOT, SQNS]",https://ml.globenewswire.com/Resource/Download...,The global IoT-based asset tracking and monito...,"[IoT, asset tracking, monitoring, manufacturin...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,SQNS,positive,SEQUANS is mentioned as an industry player in ...
3,7f5ac64e05883e51d4ec487883de5d27fdbae80dc8cf0d...,Fleet Management in the Americas Industry Repo...,Researchandmarkets.Com,2024-12-06T19:54:00Z,https://www.globenewswire.com/news-release/202...,"[VZ, IOT, TRMB, AIOT, PCAR, F, FpB, FpC, FpD, ...",https://ml.globenewswire.com/Resource/Download...,The report forecasts healthy growth in the fle...,"[fleet management, telematics, North America, ...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,VZ,positive,Verizon Connect is mentioned as one of the lea...
4,7f5ac64e05883e51d4ec487883de5d27fdbae80dc8cf0d...,Fleet Management in the Americas Industry Repo...,Researchandmarkets.Com,2024-12-06T19:54:00Z,https://www.globenewswire.com/news-release/202...,"[VZ, IOT, TRMB, AIOT, PCAR, F, FpB, FpC, FpD, ...",https://ml.globenewswire.com/Resource/Download...,The report forecasts healthy growth in the fle...,"[fleet management, telematics, North America, ...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,IOT,positive,Samsara is identified as one of the leading fl...
5,7f5ac64e05883e51d4ec487883de5d27fdbae80dc8cf0d...,Fleet Management in the Americas Industry Repo...,Researchandmarkets.Com,2024-12-06T19:54:00Z,https://www.globenewswire.com/news-release/202...,"[VZ, IOT, TRMB, AIOT, PCAR, F, FpB, FpC, FpD, ...",https://ml.globenewswire.com/Resource/Download...,The report forecasts healthy growth in the fle...,"[fleet management, telematics, North America, ...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN,TRMB,neutral,Trimble is mentioned as one of the solution pr...
6,7f5ac64e05883e51d4ec487883de5d27fdbae80dc8cf0d...,Fleet Management in the Americas Industry Repo...,Researchandmarkets.Com,2024-12-06T19:54:00Z,https://www.globenewswire.com/news-release/202...,"[VZ, IOT, TRMB, AIOT

In [7]:



API_KEY = API_KEY
ticker = "NVDA"
url = f"https://api.polygon.io/v2/aggs/ticker/IBM/range/1/day/2024-01-09/2026-02-10?adjusted=true&sort=asc&apiKey=ukAiYu4ajI043S6moLu7RJV_CGzWGbAy"

response = requests.get(url)
data = response.json()
print(data)

{'ticker': 'IBM', 'queryCount': 274, 'resultsCount': 274, 'adjusted': True, 'results': [{'v': 2617186.0, 'vw': 160.0753, 'o': 160, 'c': 160.08, 'h': 160.4837, 'l': 159.51, 't': 1704776400000, 'n': 53832}, {'v': 2967852.0, 'vw': 160.8399, 'o': 160.28, 'c': 161.23, 'h': 161.34, 'l': 159.74, 't': 1704862800000, 'n': 44998}, {'v': 3778395.0, 'vw': 161.6549, 'o': 161.02, 'c': 162.16, 'h': 162.23, 'l': 160.29, 't': 1704949200000, 'n': 50910}, {'v': 4958261.0, 'vw': 164.8162, 'o': 162.97, 'c': 165.8, 'h': 165.98, 'l': 162.355, 't': 1705035600000, 'n': 72352}, {'v': 4869635.0, 'vw': 166.4729, 'o': 165.8, 'c': 166.96, 'h': 167.25, 'l': 165.34, 't': 1705381200000, 'n': 70020}, {'v': 4288604.0, 'vw': 166.4975, 'o': 166.79, 'c': 166.08, 'h': 167.82, 'l': 165.495, 't': 1705467600000, 'n': 66939}, {'v': 3776990.0, 'vw': 166.4366, 'o': 166.49, 'c': 166.84, 'h': 166.99, 'l': 165.04, 't': 1705554000000, 'n': 57006}, {'v': 6929079.0, 'vw': 170.7205, 'o': 170.59, 'c': 171.48, 'h': 171.5791, 'l': 169.18, 

In [8]:
df = pd.DataFrame(data)
df = pd.json_normalize(df['results'])

df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
df

,v,vw,o,c,h,l,t,n,timestamp
0,2617186.0,160.0753,160.00,160.08,160.4837,159.510,1704776400000,53832,2024-01-09 05:00:00
1,2967852.0,160.8399,160.28,161.23,161.3400,159.740,1704862800000,44998,2024-01-10 05:00:00
2,3778395.0,161.6549,161.02,162.16,162.2300,160.290,1704949200000,50910,2024-01-11 05:00:00
3,4958261.0,164.8162,162.97,165.80,165.9800,162.355,1705035600000,72352,2024-01-12 05:00:00
4,4869635.0,166.4729,165.80,166.96,167.2500,165.340,1705381200000,70020,2024-01-16 05:00:00
...,...,...,...,...,...,...,...,...,...
269,6051189.0,263.2419,265.71,263.30,265.7200,261.180,1738731600000,90057,2025-02-05 05:00:00
270,5990940.0,255.8259,262.98,253.44,263.3800,252.730,1738818000000,104872,2025-02-06 05:00:00
271,3309630.0,253.2765,255.28,252.34,256.9300,252.020,1738904400000,73419,2025-02-07 05:00:00
272,3428527.0,248.8956,250.86,249.27,251.9500,246.870,1739163600000,69698,2025-02-10 05:00:00
